## Import Libraries and load in the data

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import constantsfile # constants.py
import math
from pulp import * #conda install -c conda-forge pulp
import kaleido
kaleido.__version__ #0.2.1

'0.2.1'

In [2]:
# Load in the S2F2D_FC and Supply_and_Demand data and create the dataframe
S2F2D_FC = pd.read_csv('S2F2D.csv')

### CREATE FIXED COST DATAFRAME###
FC = S2F2D_FC.loc[:,['Demand_name',
                     'Demand_tonnes', 
                     'Facility_name',
                     'Supply_name',
                     'Supply_tonnes',
                     'Fixed_cost',
                     'Transport_cost_F2D',
                     'Transport_cost_S2F']]

## Optimize

### Specify number of facilities before running the optimization

In [3]:
number_of_facilities = 3

# specify facility capacity (done in the constantsfile.py file)
total_facility_capacity = constantsfile.Facility

### Prepare a copy of the FC dataframe used for the optimization

In [4]:
# Make a copy to avoid changing the original data when changing the number of facilities
df = FC.copy(deep=True) 

# Rename some columns for easier reading
rename_dict = {
    'Demand_name': 'Customer',
    'Demand_tonnes': 'Demand',
    'Facility_name': 'Facility',
    'Supply_name': 'Supplier',
    'Supply_tonnes': 'Supply',
    }

for old_column, new_column in rename_dict.items():
    if old_column in df.columns:
        df.rename(columns={old_column: new_column}, inplace=True)
        
# Update the Fixed_cost based on the number of facilities
df.loc[:,'Fixed_cost'] = df.loc[:,'Fixed_cost']*( math.sqrt(number_of_facilities) / number_of_facilities)

### Turn dataframe data format into the correct form for the solver and perform linear programming optimization

In [5]:
# Extract actor node dictionaries
Supplier = df['Supplier'].unique().tolist()
Facility = df['Facility'].unique().tolist()
Customer = df['Customer'].unique().tolist()

# Extract capacities dictionaries
supplier_supply = df.groupby('Supplier')['Supply'].first().to_dict()
facility_capacity = total_facility_capacity/number_of_facilities # constant for each facility
customer_demand = df.groupby('Customer')['Demand'].first().to_dict()

# Extract fixed_cost dictionary
fixed_cost = df.groupby('Facility')['Fixed_cost'].first().to_dict()

# Create dictionaries for transportation costs
transportation_cost_S2F = {(a, b): c for a, b, c in df[['Supplier', 'Facility', 'Transport_cost_F2D']].values}
transportation_cost_F2D = {(a, b): c for a, b, c in df[['Facility', 'Customer', 'Transport_cost_S2F']].values}

# Setting the Problem
prob = LpProblem("Capacitated Facility Location Problem", LpMinimize)

# Defining our Decision Variables
x = LpVariable.dicts("X", ((i,j) for i in Supplier for j in Facility), lowBound=0, cat='Continuous') # Supplier-facility flow [tonnes]
y = LpVariable.dicts("Y", Facility, lowBound=0, upBound=1, cat='Integer') # Facility [0 (inactive) or 1 (active)]
z = LpVariable.dicts("Z", ((j, k) for j in Facility for k in Customer), lowBound=0, cat='Continuous') # Facility-customer flow [tonnes]

# OBJECTIVE FUNCTION

prob += (lpSum([transportation_cost_S2F[i, j] * x[i, j] for i in Supplier for j in Facility]) +
          lpSum([transportation_cost_F2D[j, k] * z[j, k] for j in Facility for k in Customer]) +
          lpSum([fixed_cost[j] * y[j] for j in Facility]))

# CONSTRAINTS

# Set number of facilities to use
prob += lpSum(y[j] for j in Facility) == number_of_facilities

# Specify maximum volumes each supplier and customer can handle to all facilities
for i in Supplier:
    prob += lpSum(x[i, j] for j in Facility) <= supplier_supply[i]
for k in Customer:
    prob += lpSum(z[j, k] for j in Facility) <= customer_demand[k]

# Only open Facility allows Customer and Supplier flows
for i in Supplier:
    for j in Facility:
        prob += x[i, j] <= facility_capacity * y[j]
for j in Facility:
    for k in Customer:
        prob += z[j, k] <= facility_capacity * y[j]

# The sum of the flows to/from each facilities is equal to its capacity, also indirectly conservation of flow
for j in Facility:
    prob += lpSum(x[i, j] for i in Supplier) == facility_capacity * y[j]
    prob += lpSum(z[j, k] for k in Customer) == facility_capacity * y[j]

prob.solve()

print("Solution Status = ", LpStatus[prob.status])

# Print the solution of Binary Decision Variables
Tolerance = 0.00001
for j in Facility:
    if y[j].varValue > Tolerance:
        print("Establish Facility at site =", j)

# # Print the solution of Continuous Decision Variables
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# Print Optimal
print("Total Cost =", value(prob.objective))

# Check flows, print results and append to lists
S = []
F =[]
D = []
flow = []

print("\nSuppliers and Facilities with flows greater than zero:")
for i in Supplier:
    for j in Facility:
        if x[i, j].varValue > Tolerance:  # checking if flow is more than a small tolerance
            print(f"Flow from Supplier {i} to Facility {j} is: {x[i, j].varValue}")
            S.append(str(i))
            F.append(str(j))
            D.append(None)
            flow.append(x[i, j].varValue)

print("\nFacilities and Customers with flows greater than zero:")
for j in Facility:
    for k in Customer:
        if z[j, k].varValue > Tolerance:  # checking if flow is more than a small tolerance
            print(f"Flow from Facility {j} to Customer {k} is: {z[j, k].varValue}")
            S.append(None)
            F.append(str(j))
            D.append(str(k))
            flow.append(x[i, j].varValue)
            
# Create a new DataFrame from site, facility, and flow and save to csv file
optimization_results_df = pd.DataFrame({'Supplier': S, 'Facility': F, 'Customer': D, 'Flow': flow})
optimization_results_df.to_csv('optimization_results.csv', index=False)

c:\Users\erik3040\AppData\Local\miniconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solution Status =  Infeasible
Establish Facility at site = N133
Establish Facility at site = N314
Establish Facility at site = N358
Establish Facility at site = N395
Total Cost = 5670543.138636574

Suppliers and Facilities with flows greater than zero:
Flow from Supplier Vienna to Facility N395 is: 44.96441
Flow from Supplier Antwerp to Facility N395 is: 27.13474
Flow from Supplier Sofia to Facility N395 is: 108.10191
Flow from Supplier Zagreb to Facility N395 is: 188.81972
Flow from Supplier Prague to Facility N395 is: 154.9621
Flow from Supplier Copenhagen to Facility N395 is: 44.500466
Flow from Supplier Tallinn to Facility N395 is: 102.68447
Flow from Supplier Helsinki to Facility N395 is: 2.1361548
Flow from Supplier Paris to Facility N395 is: 196.25659
Flow from Supplier Marseille to Facility N395 is: 144.24976
Flow from Supplier Berlin to Facility N395 is: 179.26618
Flow from Supplier Cologne to Facility N395 is: 111.99353
Flow from Supplier Düsseldorf to Facility N395 is: 17.58

# Visualize the result

by creating a new dataframe from the optimization results and merge selected nodes to the coordinates from the S2F2D dataframe

In [6]:
# For the visualization Create a new dataframe with a column of unique instances of supplier, faciliy and customer from the optimization_results dataframe

# Unpivot the DataFrame
unpivoted_df = pd.melt(optimization_results_df, 
                       id_vars=[], 
                       value_vars=['Supplier', 
                                   'Facility', 
                                   'Customer'], 
                       var_name='Type', 
                       value_name='Name')

# Remove duplicates and None values
unpivoted_df = unpivoted_df.drop_duplicates().dropna().reset_index(drop=True)

Match the results dataframe (unpivoted_df) to the coordinates for each node in the S2F2D_FC dataframe

In [7]:
# Get unique coordinates for all the nodes in the S2F2D_FC dataframe
unique_demand_coordinates = S2F2D_FC[['Demand_name', 'Demand_latitude', 'Demand_longitude']].drop_duplicates().reset_index(drop=True)
unique_supply_coordinates = S2F2D_FC[['Supply_name', 'Supply_latitude', 'Supply_longitude']].drop_duplicates().reset_index(drop=True)
unique_facility_coordinates = S2F2D_FC[['Facility_name', 'Facility_latitude', 'Facility_longitude']].drop_duplicates().reset_index(drop=True)

# Rename the columns to match the unpivoted_df_merged dataframe
unique_demand_coordinates.rename(columns={'Demand_name': 'Name',
                                          'Demand_latitude': 'Latitude',
                                          'Demand_longitude': 'Longitude'}, inplace=True)

unique_supply_coordinates.rename(columns={'Supply_name': 'Name',
                                            'Supply_latitude': 'Latitude',
                                            'Supply_longitude': 'Longitude'}, inplace=True)

unique_facility_coordinates.rename(columns={'Facility_name': 'Name',
                                            'Facility_latitude': 'Latitude',
                                            'Facility_longitude': 'Longitude'}, inplace=True)

# Add a new columnn to the unique_demand_coordinates dataframes to indicate the type of node
unique_demand_coordinates['Type'] = 'Customer'
unique_supply_coordinates['Type'] = 'Supplier'
unique_facility_coordinates['Type'] = 'Facility'

# Merge the unique_demand_coordinates, unique_supply_coordinates, and unique_facility_coordinates dataframes
unique_all_coordinates = pd.concat([unique_demand_coordinates, unique_supply_coordinates, unique_facility_coordinates], ignore_index=True)

# Merge the unpivoted_df and unpivoted_df_merged dataframes
unpivoted_df_merged = pd.merge(unpivoted_df, unique_all_coordinates, how='left', on=['Name', 'Type'])
unpivoted_df_merged

,Type,Name,Latitude,Longitude
0,Supplier,Vienna,48.212196,16.376599
1,Supplier,Antwerp,51.212183,4.408770
2,Supplier,Sofia,42.696672,23.334975
3,Supplier,Zagreb,45.794975,15.979962
4,Supplier,Prague,50.075665,14.430168
...,...,...,...,...
99,Customer,Northvolt–Volvo Cars,57.711823,11.931610
100,Customer,Rosatom,54.646200,20.545089
101,Customer,AMTE Power,56.480303,-2.952726
102,Customer,West Midlands,52.377769,-1.500484


Adjust marker size

In [8]:
category_to_size = {
    'Facility': 50,
    'Supplier': 10,
    'Customer': 20,
}

unpivoted_df_merged['marker_size'] = unpivoted_df_merged['Type'].map(category_to_size)

Display results in figure

In [9]:
fig = px.scatter_mapbox(unpivoted_df_merged,
                        lat ="Latitude",
                        lon="Longitude",
                        color="Type",
                        category_orders={'Type':list(unpivoted_df_merged['Type'].unique())},
                        color_discrete_sequence=['#5ba300','#000000','#e6308a'],
                        zoom=2.3, center=dict(lon=15, lat=56),
                        height=600, width=800,
                        size="marker_size",
                        opacity=0.8,
                        color_continuous_scale=px.colors.sequential.Bluered,)
fig.update_layout(mapbox_style='light', mapbox_accesstoken=constantsfile.mapbox_access_token)
fig.update_layout(title_text="Optimal <span style='color: #5ba300; font-weight: bold;'>Supply</span>, <span style='color: #000000; font-weight: bold;'>Facility</span> and <span style='color: #e6308a; font-weight: bold;'>Demand</span> placements.")
fig.update_layout(showlegend=False)
fig.show()
#fig.write_image("optimization_results.png" , scale=1, engine='kaleido')